# The Battle of Neighbourhoods 

## Introduction and context
In this (hypothetical) problem a friend of mine has quit his Wall Street job after having worked for a bank for 10+ years. Wanting a change from corporate culture and having a bit of spare money to invest, but unable to leave the City he’s looking into opening a Hungarian restaurant in New York. While most of his friends think this is at least risky if not borderline madness, New York is probably the only and best place to open a place with a less known and somewhat less health-conscious cuisine. 

New York has more than 8.5 million people, while the greater metropolitan area’s population is more than 23 million, with neighborhoods home to an extremely vast majority of places from eateries through hole in the wall places up to 2-3 michelin star fine dining restaurants. According to the New York City Department of Health and statista (https://www.statista.com/statistics/259776/number-of-people-who-went-to-restaurants-in-new-york-by-type/)  there were more than 26’000 restaurants in the City in 2017, this gives a glimpse of hope for having yet another obscure place to make ends meet. 

The ask was to analyze where it may make sense to open such a restaurant. New York is very diverse with no very obvious concentration of cuisines (apart from Chinatown and some Flushing Meadows districts), so it requires further analysis to see if there is a trend in concentration of small cuisines or maybe even Easter European block in places. 

### Business problem
There are a few ways to approach this problem, I’ll take 3 here and based on the later data analysis it may be possible to pick one (or may not). The approaches are:

#### Chinatown approach
New York’s Chinatown is one the largest of its kind with a massive number of prospering restaurants. Obviously not only residents and people of Chinese origin visit these, but is famous among visitors, tourists and in general as well. This approach assumes that if there are areas with concentrated Hungarian and in a broader sense Eastern European restaurants another one can still fit in as people do visit these parts to eat a particular dish. As Hungarian is a small portion of the city's population with a lesser known cuisine it does seem to make sense to extend the radius with similar cuisines as well. (admittedly with a subjective list)

#### Go against the current
This is the direct opposite - seeing if there are places with no or very limited number of similar restaurants. In other cities this may be a plain bad approach as purely residential areas, suburbs or other industrial districts would not be  a good fit, but NYC, especially Manhattan is so packed with restaurants that this may not be an issue there. However, if such area is found, it does make sense to see how many / how concentrated the place is.

#### Go with the flow
This approach will simply look at the areas with the highest concentration of restaurants (assuming it also is proportional to their variety) and will recommend to set up a place where there are already a lot, as people do go there to eat and is a well-established neighbourhood from this point of view. 

More specifically during the exercise the following questions will be answered:
What are the particular areas with high concentration of Hungarian restaurants?
In a more generic sense, what are the areas with high concentration of Eastern European (Hungarian, Czech, Slovakian, Polish, Romanian) cuisines?
Which areas do not have Eastern European restaurants?
What are the areas with the highest concentration of restaurants?

## Description of the data and how it’ll be used
I will essentially be using the same data sets from the previous week’s exercises, as follows:

 
### New York neighbourhood data, latitude, longitude information
Source: https://cocl.us/new_york_dataset
Usage: mapping restaurant data including address to borough and neighbourhood for classification 

### Hungarian and Eastern European restaurants in New York City
Source: Foursquare API
Usage: getting the list of Hungarian and Eastern European restaurants in NYC for each neighbourhood

### GeoSpacial data
Source: https://data.cityofnewyork.us/City-Government/Borough-
Boundaries/tqmj-j8zm
Usage: using for neighbourhood boundaries for visualization

## Methodology
For this particular example the methodology is quite straightforward and analyzing the data is not overly complex either. The steps to achieve the desired answers and to draw some conclusions (if possible) are the following:
Gather and clean / scrape New York neighbourhood data
Read data from https://cocl.us/new_york_dataset
Read and map geo data (latitude and longitude) to each Neighborhood and borough
Read the list of Hungarian and Eastern European restaurants from the Foursquare API mainly focusing on Manhattan
Do some exploratory data analysis to see if the scope of the analysis could be or should be changed (i.e.) remove Neighborhoods are   
Run a simple density analysis on neighbourhoods to see which ones have the highest concentration of restaurants
Use the K-Means method to cluster neighbourhoods and get answers for the distribution of Eastern European restaurants. 
